#### IMPORT LIBRARIES

In [1]:
import configparser
from typing import List
import tweepy
import pandas as pd
import psycopg2

#### READ CONFIG

In [2]:
# read config 
config = configparser.ConfigParser()
config.read('config.ini')

# read Twitter Config
CONSUMER_KEY = config['twitter']['CONSUMER_KEY']
CONSUMER_SECRET = config['twitter']['CONSUMER_SECRET']
ACCESS_TOKEN = config['twitter']['ACCESS_TOKEN']
ACCESS_SECRET = config['twitter']['ACCESS_SECRET']

LAST_TWEET_ID = config['filename']['LAST_TWEET_ID_FILENAME']

# read database password
DATABASE_PASSWORD = config['database']['DATABASE_PASSWORD']

#### AUTHENTICATE TWITTER

In [3]:
# authenticate
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

#### CONNECT DATABASE

In [25]:
conn = psycopg2.connect("host=localhost dbname=tweets user=postgres password=example")

#### BUSINESS LOGIC

In [26]:
def get_tweets(keyword: str, lastTweetId) -> List[str]:
  all_tweets = []

  # for tweet in api.search_tweets(q=keyword, lang='en', count=10, result_type="mixed", tweet_mode="extended"):
  for tweet in api.search_tweets(q=keyword, lang='en', count=10, result_type="mixed", tweet_mode="extended", since_id=lastTweetId):
    all_tweets.append(tweet)
  
  return all_tweets

# def getPremiumTweets(keyword: str) -> List[str]:
#   premiumTweets = []
#   for tweet in api.search_30_day(label="development", query=keyword, maxResults=10):
#     premiumTweets.append(tweet)
#   return premiumTweets

def parsingTweets(tweets: List) -> List[str]:
  parsedTweets = []

  for tweet in tweets:
    parsedTweets.append({
      "created_at": tweet._json["created_at"],
      "id": tweet._json["id"],
      "full_text": tweet._json["full_text"],
      "username": tweet._json["user"]["name"],
      "handle": tweet._json["user"]["screen_name"],
      "location": tweet._json["user"]["location"],
      "favorite_count": tweet._json["favorite_count"],
      "retweet_count": tweet._json["retweet_count"]
    })
    
  return parsedTweets


def saveLastTweetId(tweets: List):
  id = tweets[-1]["id"]

  fileName = LAST_TWEET_ID
  f = open(fileName, 'w+')
  f.write(str(id))
  f.close()
  return None

def readLastTweetId():
  fileName = LAST_TWEET_ID
  f = open(fileName, "r")
  id = int(f.read())
  return id

tweets = get_tweets('covid19', readLastTweetId())
parsedTweets = parsingTweets(tweets)
saveLastTweetId(parsedTweets)

tweetsDf = pd.DataFrame.from_dict(parsedTweets)
# print(tweetsDf.head(10))
tweetsDf

,created_at,id,full_text,username,handle,location,favorite_count,retweet_count
0,Thu Dec 08 19:19:18 +0000 2022,1600933052019974144,RT @nighealthwatch: In a public health emergen...,Pirates 🏴‍☠️,EsDeDiariokoin,,0,8
1,Thu Dec 08 19:19:11 +0000 2022,1600933022995337216,RT @_CEPHRC: Your #DigitalRights and #HumanRig...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,12
2,Thu Dec 08 19:19:03 +0000 2022,1600932989260554241,RT @org_scp: In August 2020 it was becoming in...,Debbie,Debbie84641181,,0,21
3,Thu Dec 08 19:19:00 +0000 2022,1600932979257532417,RT @_CEPHRC: Your #HealthRecords and #DNA can ...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,14
4,Thu Dec 08 19:18:51 +0000 2022,1600932940938366977,RT @_CEPHRC: #Covid19 hoax #Genocide stopped i...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,12
5,Thu Dec 08 19:18:39 +0000 2022,1600932890011107328,RT @_FairElection: #Depopulation By #Plandemic...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,11
6,Thu Dec 08 19:18:31 +0000 2022,1600932856012079104,RT @SCBriand: @kakape highlights the importanc...,"Clement Haeck PhD, PharmD, MS 🇪🇺🇫🇷",ClemHaeck,"New York City, 🇺🇸. Own views",0,3
7,Thu Dec 08 19:18:19 +0000 2022,1600932804837408768,RT @_FairElection: Both #Covid19 and #GlobalWa...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,13
8,Thu Dec 08 19:18:09 +0000 2022,1600932765968789504,RT @_FairElection: People have woken up and th...,Indian Elections And Political Dramas Of India...,CongressOrBJP,"New Delhi, India",0,13
9,Thu Dec 08 19:18:09 +0000 2022,1600932765549174784,Article summary: https://t.co/kt6NYCmTdG (I'm ...,Summarizer (by Otherweb),valurank,"Austin, TX",0,0
